In [3]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction to CSE142

## To sort or not to sort?

Consider the following code snippet, what the performance looks like when option is set to 1 or 0?

In [4]:
render_code("calculate_sum.c")

// calculate_sum.c:1-15 (15 lines)
#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold)
{
    long long sum=0;
    for (unsigned i = 0; i < size; ++i) {
        if (data[i] >= threshold)
                sum ++;
    }
    return sum;
}
#ifdef __cplusplus
}
#endif

In [5]:
render_code("arraySort.cpp")

// arraySort.cpp:1-37 (37 lines)
#include <algorithm>
#include <ctime>
#include <iostream>
#include <climits>

#ifdef __cplusplus
extern "C" {
#endif
long long calculate_sum(int *data, unsigned int size, int threshold);
#ifdef __cplusplus
}
#endif


int main(int argc, char **argv)
{
    // generate data
//    const unsigned arraySize = 262144;
    int arraySize = atoi(argv[1]);
    int data[arraySize];
    long long sum = 0;

    int iterations = atoi(argv[2]);
    int option = atoi(argv[3]);

    for (unsigned c = 0; c < arraySize; ++c)
        data[c] = std::rand();

    if(option)
        std::sort(data, data + arraySize);

    for (unsigned i = 0; i < iterations; ++i) {
        sum += calculate_sum(data, arraySize, std::rand());
    }
    std::cout << "sum = " << sum << std::endl;
}

In [6]:
! make clean; make

rm -f madd arraySort bitonic quicksort arraySortO3
gcc madd.c -o madd 
gcc calculate_sum.c -c 
g++ -O0 arraySort.cpp calculate_sum.o -o arraySort
/usr/local/cuda/bin/nvcc -O3 bitonic.cu -o bitonic
/usr/local/cuda/bin/nvcc -O3 quicksort.cu -o quicksort
gcc -O3 calculate_sum.c -c -o calculate_sum_O3.o
g++ -O3 arraySort.cpp calculate_sum_O3.o -o arraySortO3


In [7]:
! time ./arraySort 262144 1000 1

sum = 127161197

real	0m0.160s
user	0m0.159s
sys	0m0.000s


In [8]:
! time ./arraySort 262144 1000 0

sum = 127161197

real	0m0.572s
user	0m0.568s
sys	0m0.004s


## Algorithm on GPUs?
Consider quicksort and bitsort, which performs better on GPUs?

### Quick sort

In [ ]:
render_code("quicksort.cu", show="quickSortIterative")

In [ ]:
render_code("quicksort.cu", show="partition")

### Bitonic sort

In [ ]:
render_code("bitonic.cu", show="bitonic_sort")

In [ ]:
render_code("bitonic.cu", show="bitonic_sort_step")

In [ ]:
! ./bitonic

In [ ]:
!./quicksort

In [1]:
! nvidia-smi -a


==============NVSMI LOG==============

Timestamp                                 : Mon Aug  7 13:19:02 2023
Driver Version                            : 535.54.03
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Product Name                          : NVIDIA GeForce RTX 4070
    Product Brand                         : GeForce
    Product Architecture                  : Ada Lovelace
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    Addressing Mode                       : None
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial N